In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import datetime
from pathlib import Path
import pandas as pd
import pandas_read_xml as pdx
from pandas_profiling import ProfileReport
from anateldb.parser import *
from anateldb.query import *

In [ ]:
SIGLAS = {"Acre" : "AC",
          "Alagoas" : "AL",
          "Amapá": "AP",
          "Amazonas":"AM",
          "Bahia":"BA",
          "Ceará":"CE",
          "Espírito Santo":"ES",
          "Goiás":"GO",
          "Maranhão":"MA",
          "Mato Grosso":"MT",
          "Mato Grosso do Sul":"MS",
          "Minas Gerais": "MG",
          "Pará": "PA",
          "Paraíba":"PB",
          "Paraná":"PR",
          "Pernambuco":"PE",
          "Piauí":"PI",
          "Rio de Janeiro":"RJ",
          "Rio Grande do Norte":"RN",
          "Rio Grande do Sul":"RS",
          "Rondônia":"RO",
          "Roraima":"RR",
          "Santa Catarina":"SC",
          "São Paulo":"SP",
          "Sergipe":"SE",
          "Tocantins":"TO",
          "Distrito Federal":"DF"}

In [ ]:
pb = read_plano_basico('D:\OneDrive - ANATEL\GR01FI3\BaseDados\plano_basico.zip')
pb.tail()

,Serviço,Status,Entidade,Fistel,Município,UF,Id,CNPJ,Frequência,Classe,Latitude,Longitude,Canal,País
79895,RTVD,TV-L9,TELEVISAO CENTRO AMERICA LTDA,50415260361,Barra do Garças,MT,60788db82eb35,03476876000105,605,C,"-15,8804","-52,264",36,BRA
79896,RTVD,TV-L21,RADIO E TV PORTOVISÃO LTDA,50416192394,Garibaldi,RS,60788dc532c1b,87209250000114,581,C,"-29,259","-51,5352",32,BRA
79897,RTV,TV-L21,TV NEWS - CANAL BRASILEIRO DE COMUNICACAO LTDA,50411694545,Montes Claros,MG,60789444f10ea,17962660000156,689,C,"-16,73667","-43,85056",50,BRA
79898,RTV,TV-L21,TELEVISAO SOCIEDADE LIMITADA,50411667068,Abaeté,MG,6078b2257cc75,25288333000199,189,C,"-19,16","-45,445833333333",9,BRA
79899,RTV,TV-L12,TELEVISAO CACHOEIRA DO SUL LTDA,50400813750,Ouro Fino,MG,6078ddfa7fce5,89784037000161,57,C,<NA>,<NA>,2,BRA


In [ ]:
# pb_profile = ProfileReport(pb, minimal=True, dark_mode=True, title='Perfil Plano Básico - Mosaico')
# pb_profile

In [ ]:
es = read_estações('D:\OneDrive - ANATEL\GR01FI3\BaseDados\estações.zip')
es.tail()

,Serviço,Status,Entidade,Fistel,Município,UF,Id,Número da Estação,CNPJ
28048,RTVD,TV-C2,FUNDACAO EDUCACIONAL E CULTURAL DE IPANEMA,50433856696,Sabará,MG,5f32c1c918e6b,,04608796000110
28049,FM,FM-C2,EMPRESA PAULISTA DE RADIODIFUSAO LTDA,50437944611,Regente Feijó,SP,5f36a03433642,,44872109000194
28050,FM,FM-C2,RADIO ITAPIRANGA LTDA,50433860456,Itapiranga,SC,5f68d432841a5,,84375872000124
28051,FM,FM-C1,EMISSORAS SUL BRASILEIRAS LTDA,50433937009,Horizontina,RS,5f8dcc96f23f9,,95818506000119
28052,FM,FM-C4,VIP RADIO E TELEVISAO LTDA,02030455644,Itanhaém,SP,6094de98db4d9,9156232,67751495000161


In [ ]:
# es_profile = ProfileReport(es, minimal=True, dark_mode=True, title='Perfil Estações - Mosaico')
# es_profile

In [ ]:
RIGHT = ['@id', '@fistel', '@SiglaServico', '@state']
LEFT = ['@id', '@Fistel', '@Servico', '@Status']
ON = ['Id', 'Fistel', 'Serviço', 'Status']
df = pb.merge(es, on='Id', how='outer') #.drop(RIGHT, axis=1)
df.shape

(82539, 22)

In [ ]:
COLS = ['Serviço', 'Status', 'Entidade', 'Fistel', 'Município', 'UF', 'CNPJ']
for col in COLS:
    col_x = col + '_x'
    col_y = col + '_y'
    if df[col_x].count() > df[col_y].count():
        a,b = col_x, col_y
    else:
        a,b = col_y, col_x
        
    df.loc[df[a].isna(), a] = df.loc[df[a].isna(), b]
    df.drop(b, axis=1, inplace=True)
    df.rename({a: a[:-2]}, axis=1, inplace=True)
df = df[df.Frequência.notna()].reset_index(drop=True)


In [ ]:
m = pd.read_excel('D:\OneDrive - ANATEL\GR01FI3\BaseDados\municípios.xlsx')

In [ ]:
df['Coordenadas_do_Município'] = False

In [ ]:
"São Paulo".encode('ascii')

UnicodeEncodeError: 'ascii' codec can't encode character '\xe3' in position 1: ordinal not in range(128)

In [ ]:
df = df[df.País == 'BRA'].reset_index(drop=True)
df['Latitude'] = df.Latitude.str.replace(',', '.')
df['Longitude'] = df.Longitude.str.replace(',', '.')
df['Frequência'] = df.Frequência.str.replace(',', '.')

In [ ]:
m['Município'] = m.Município.apply(lambda x: ''.join(e for e in x if e.isalnum()))

In [ ]:
from tqdm.auto import tqdm
for row in tqdm(df[df.Latitude.isna()].itertuples()):
    left = ''.join(e for e in row.Município if e.isalnum())
    df.loc[row.Index, ['Latitude', 'Longitude']] = m.loc[(m.Município == left) & (m.UF == row.UF), ['Latitude', 'Longitude']].values.flatten().tolist()    
    df.loc[row.Index, 'Coordenadas_do_Município'] = True
df['Latitude'] = df.Latitude.astype('float')
df['Longitude'] = df.Longitude.astype('float')
df['Frequência'] = df.Frequência.astype('float')

0it [00:00, ?it/s]

In [ ]:
row.Município

'Olho d Água das Cunhãs'

In [ ]:
m.loc[(m.Município == row.Município) & (m.UF == row.UF), ['Latitude', 'Longitude']].values.flatten().tolist()

In [ ]:
m[m.Município.str.contains('Cunhãs')]

In [ ]:
# from geopy.geocoders import Nominatim
# from geopy import *
# geolocator = Nominatim(user_agent='anateldb')

# for row in tqdm(df[df.Município.isna()].itertuples()):
#     loc = geolocator.reverse((row.Latitude,row.Longitude))
#     df.loc[row.Index, 'UF'] = SIGLAS[loc.raw.get('address').get('state')]
#     df.loc[row.Index, 'Município'] = loc.raw.get('address').get('town')
    
#     elif not loc:
#         df.loc[row.Index, 'UF'] = row.País
#         df.loc[row.Index, 'Município'] = row.País            
#     else:
#         if (address := loc.raw.get('address')):
#             if uf := address.get('state'):
#                 df.loc[row.Index, 'UF'] = uf
#             else:
#                 df.loc[row.Index, 'UF'] = row.País
#             if cidade := address.get('city'):
#                 df.loc[row.Index, 'Município'] = cidade
#             else:
#                 df.loc[row.Index, 'Município'] = row.País            

In [ ]:
# merge_profile = ProfileReport(df, minimal=True, dark_mode=True, title='Perfil Estações + Plano Básico - Mosaico')
# merge_profile

In [ ]:
df.to_feather('D:\OneDrive - ANATEL\GR01FI3\BaseDados\mosaico.fth')

In [ ]:
df = pd.read_feather('D:\OneDrive - ANATEL\GR01FI3\BaseDados\mosaico.fth')

In [ ]:
df.Serviço.unique()

In [ ]:
# profile = ProfileReport(df, minimal=True, dark_mode=True, title='Perfil Mosaico')
# profile

In [ ]:
radcom = read_radcom('D:\OneDrive - ANATEL\GR01FI3\BaseDados')

In [ ]:
radcom.head()

In [ ]:
radcom['Serviço'] = '231'
radcom['Status'] = 'RADCOM'
radcom['Classe'] = radcom.Fase.str.strip() + '-' + radcom.Situação.str.strip()
radcom['Entidade'] = radcom.Entidade.str.rstrip().str.lstrip()

In [ ]:
rd = df.append(radcom).sort_values("Frequência").reset_index(drop=True) ; rd.tail()

In [ ]:
rd.loc[rd.Status.str.contains('AM'), 'Frequência'] = rd.loc[rd.Status.str.contains('AM'), 'Frequência'] / 1000 ; rd.loc[rd.Status.str.contains('AM'), 'Frequência'].head()

In [ ]:
rd = rd.fillna('-') ; rd.tail()

In [ ]:
rd.shape

In [ ]:
df = pd.DataFrame(columns=rd.columns)
for s in ['C1', 'C2', 'C3', 'C4', 'C7', 'RADCOM']:
    df = df.append(rd[rd.Status.str.contains(s)])
df = df.reset_index(drop=True); df.shape

In [ ]:
rd = df

In [ ]:
rd['Description'] = rd.Status + ', ' + rd.Classe + ', ' + rd.Entidade.str.title() + ' (' + \
                                rd.Fistel.astype('str') + ', ' + rd['Número da Estação'].astype('str') + '), ' + rd.Município + '/' + \
                                rd.UF

In [ ]:
rd.loc[rd.Coordenadas_do_Município == True, 'Description'] = rd.loc[rd.Coordenadas_do_Município == True, 'Description'] + '*'

In [ ]:
rd.head()

In [ ]:
exp = rd[['Frequência', 'Latitude', 'Longitude', 'Description']].copy()
exp.columns = ['Frequency', 'Latitude', 'Longitude', 'Description']

In [ ]:
stel = read_stel('D:\OneDrive - ANATEL\GR01FI3\BaseDados').sort_values('Frequência').loc[:, TELECOM]

In [ ]:
stel.Serviço.unique()

In [ ]:
slma = (stel.Frequência >= 108) & (stel.Frequência <= 137)
telecom = stel[~slma].reset_index(drop=True)
slma = stel[slma].reset_index(drop=True)

In [ ]:
slma['Description'] = slma.Serviço + ', ' + slma.Entidade.str.title() + ' (' + slma.Fistel.astype('str') + ', ' +  slma['Número da Estação'].astype('str') + '), ' + \
                        slma['Município'] + '/' + slma.UF
telecom['Description'] = telecom.Serviço + ', ' + telecom.Entidade.str.title() + ' (' + telecom.Fistel.astype('str') + ', ' + telecom['Número da Estação'].astype('str') + '), ' + \
                           telecom['Município'] + '/' + telecom.UF

In [ ]:
slma = slma[['Frequência', 'Latitude', 'Longitude', 'Description']]
slma.columns = ['Frequency', 'Latitude', 'Longitude', 'Description']

In [ ]:
telecom = telecom[['Frequência', 'Latitude', 'Longitude', 'Description']]
telecom.columns = ['Frequency', 'Latitude', 'Longitude', 'Description']

In [ ]:
exp = exp.sort_values('Frequency')

In [ ]:
sub = {' Da ': ' da ', ' De ': ' de ', ' Do ': ' do ',
       ' Das ': ' das ', ' Dos ': ' dos ', ' E ': ' e '}

In [ ]:
for k,v in sub.items():
    exp['Description'] = exp.Description.str.replace(k,v)
    slma['Description'] = slma.Description.str.replace(k,v)
    telecom['Description'] = telecom.Description.str.replace(k,v)
    

In [ ]:
exp['Description'] = exp.Description.str.replace('\s{2,}', ' ', regex=True)
slma['Description'] = slma.Description.str.replace('\s{2,}', ' ', regex=True)
telecom['Description'] = telecom.Description.str.replace('\s{2,}', ' ', regex=True)    

In [ ]:
dest = r'D:\OneDrive - ANATEL\GR01FI3\BaseDados\FiltroAppAnalise'

for p in Path(dest).iterdir():
    p.unlink()

time = datetime.today().strftime("%Y%m%d_T%H%M%S")

with pd.ExcelWriter(f'{dest}/AnatelDB_{time}.xlsx') as workbook:
    exp.to_excel(workbook, sheet_name='Sheet1', index=False)
    slma.to_excel(workbook, sheet_name='Sheet2', index=False)
    telecom.to_excel(workbook, sheet_name='Sheet3', index=False)

In [ ]:
rd.groupby('Status').count()['Serviço'].sort_values(ascending=False)

In [ ]:
# from tqdm.notebook import tqdm
# geolocator = Nominatim(user_agent='anateldb')

# from geopy.extra.rate_limiter import RateLimiter
# geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)

# def info_from_coord(row):
#     loc = geocode((row.Latitude,row.Longitude))
#     if not hasattr(loc, 'raw'):
#         return row.País, row.País
#     if (address := loc.raw.get('address')):
#         if not (uf := address.get('state')): 
#             uf = row.País             
#         if not (cidade := address.get('city')):
#             if not (cidade := address.get('town')):
#                 cidade = row.País
#         return cidade, uf
#     return row.País, row.País